In [20]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from sklearn import linear_model
import seaborn as sns
import geopandas as gpd


In [2]:
con = sqlite3.connect("MEJ.db")

In [5]:
exposure_indicators = pd.read_sql("SELECT * FROM exposure;", con)
exposure_indicators.head()

,index,Tract_ID,PTRAF,PTRAF_rank,ozone,ozone_rank,PM25,PM25_rank,diesel,diesel_rank,toxics,toxics_rank,cancer,cancer_rank
0,0,08001007801,993.842475,84.787830,51.944987,41.393114,8.062005,87.029624,0.402218,83.767228,0.045810,83.856159,34.0813,82.631982
1,1,08001007802,1338.154332,89.831865,51.970225,41.713371,8.053269,86.709367,0.505894,92.189893,0.047580,87.222647,33.4908,81.331293
2,2,08001007900,107.061809,26.821457,51.917970,40.352282,8.115519,88.150520,0.399749,83.154671,0.044924,83.091048,33.1968,80.107116
3,3,08001008000,337.873777,50.600480,51.924142,40.832666,8.099520,87.990392,0.494839,91.424196,0.047004,86.074981,32.7842,79.035960
4,4,08001008100,1853.973994,93.755004,51.930978,41.072858,8.061039,86.949560,0.525046,93.261868,0.048930,88.293803,33.2987,80.566182


In [12]:
exposure_indicators.shape

(1249, 14)

In [9]:
sf_inds = pd.read_sql("SELECT * FROM socioeconomic_factors;", con)
sf_inds.head()

,index,Tract_ID,house_burden,house_burden_rank,total_pop,poverty,poverty_rank,edu,edu_rank,ling_iso,ling_iso_rank,unemploy,unemploy_rank,nonwhite,nonwhite_rank
0,0,08001007801,0.313725,98.049194,4137.0,62.073967,97.497982,46.6,99.917966,51.7,100.000000,2.852308,65.384615,85.3,98.546042
1,1,08001007802,0.226022,92.281595,4664.0,69.425386,99.354318,45.8,99.835931,39.5,99.828179,3.451973,78.290598,86.5,98.949919
2,2,08001007900,0.233983,93.638677,5684.0,55.365939,94.511703,33.8,97.169811,28.6,98.539519,3.307530,75.897436,77.4,97.092084
3,3,08001008000,0.183908,81.340119,6241.0,53.981734,93.462470,29.2,95.488105,26.3,97.594502,2.627784,60.341880,78.5,97.536349
4,4,08001008100,0.321481,98.218830,1525.0,38.950820,78.288943,11.2,72.682527,11.6,84.922680,1.377049,21.965812,42.2,76.009693


In [13]:
sf_inds.shape

(1249, 15)

In [8]:
census_tracts = pd.read_sql("SELECT * FROM census_tracts;", con)
census_tracts.head()

,Tract_ID,NAME,geometry
0,8043979000,"Census Tract 9790, Fremont County, Colorado",POLYGON ((-105.8835319999999 38.32816700000006...
1,8045951600,"Census Tract 9516, Garfield County, Colorado","POLYGON ((-107.432269 39.57173400000005, -107...."
2,8069002803,"Census Tract 28.03, Larimer County, Colorado","POLYGON ((-105.579058 40.33209400000004, -105...."
3,8125963200,"Census Tract 9632, Yuma County, Colorado","POLYGON ((-102.803549 39.99325000000005, -102...."
4,8069002401,"Census Tract 24.01, Larimer County, Colorado","POLYGON ((-105.220725 40.79378600000007, -105...."


In [19]:
census_tracts['geometry']

0       POLYGON ((-105.8835319999999 38.32816700000006...
1       POLYGON ((-107.432269 39.57173400000005, -107....
2       POLYGON ((-105.579058 40.33209400000004, -105....
3       POLYGON ((-102.803549 39.99325000000005, -102....
4       POLYGON ((-105.220725 40.79378600000007, -105....
                              ...                        
1244    POLYGON ((-104.826874 39.71730100000008, -104....
1245    POLYGON ((-104.860149 39.68204000000003, -104....
1246    POLYGON ((-104.8435899999999 39.65844800000008...
1247    POLYGON ((-104.829953 39.69661900000006, -104....
1248    POLYGON ((-104.828840999 39.70408300000003, -1...
Name: geometry, Length: 1249, dtype: object

In [23]:
from shapely import wkt

census_tracts['geometry'] = census_tracts['geometry'].apply(wkt.loads)
census_tracts_geo = gpd.GeoDataFrame(census_tracts, geometry='geometry')

In [24]:
census_tracts_geo

,Tract_ID,NAME,geometry
0,8043979000,"Census Tract 9790, Fremont County, Colorado","POLYGON ((-105.88353 38.32817, -105.88385 38.3..."
1,8045951600,"Census Tract 9516, Garfield County, Colorado","POLYGON ((-107.43227 39.57173, -107.43309 39.5..."
2,8069002803,"Census Tract 28.03, Larimer County, Colorado","POLYGON ((-105.57906 40.33209, -105.57910 40.3..."
3,8125963200,"Census Tract 9632, Yuma County, Colorado","POLYGON ((-102.80355 39.99325, -102.80359 39.9..."
4,8069002401,"Census Tract 24.01, Larimer County, Colorado","POLYGON ((-105.22073 40.79379, -105.22117 40.7..."
...,...,...,...
1244,8005081000,"Census Tract 810, Arapahoe County, Colorado","POLYGON ((-104.82687 39.71730, -104.82687 39.7..."
1245,8005080300,"Census Tract 803, Arapahoe County, Colorado","POLYGON ((-104.86015 39.68204, -104.85898 39.6..."
1246,8005080400,"Census Tract 804, Arapahoe County, Colorado","POLYGON ((-104.84359 39.65845, -104.84377 39.6..."
1247,8005080800,"Census Tract 808, Arapahoe County, Colorado","POLYGON ((-104.82995 39.69662, -104.83047 39.6..."


In [25]:
census_tracts_geo_df = gpd.GeoDataFrame(census_tracts_geo, geometry='geometry')

In [26]:
census_tracts_geo_df

,Tract_ID,NAME,geometry
0,8043979000,"Census Tract 9790, Fremont County, Colorado","POLYGON ((-105.88353 38.32817, -105.88385 38.3..."
1,8045951600,"Census Tract 9516, Garfield County, Colorado","POLYGON ((-107.43227 39.57173, -107.43309 39.5..."
2,8069002803,"Census Tract 28.03, Larimer County, Colorado","POLYGON ((-105.57906 40.33209, -105.57910 40.3..."
3,8125963200,"Census Tract 9632, Yuma County, Colorado","POLYGON ((-102.80355 39.99325, -102.80359 39.9..."
4,8069002401,"Census Tract 24.01, Larimer County, Colorado","POLYGON ((-105.22073 40.79379, -105.22117 40.7..."
...,...,...,...
1244,8005081000,"Census Tract 810, Arapahoe County, Colorado","POLYGON ((-104.82687 39.71730, -104.82687 39.7..."
1245,8005080300,"Census Tract 803, Arapahoe County, Colorado","POLYGON ((-104.86015 39.68204, -104.85898 39.6..."
1246,8005080400,"Census Tract 804, Arapahoe County, Colorado","POLYGON ((-104.84359 39.65845, -104.84377 39.6..."
1247,8005080800,"Census Tract 808, Arapahoe County, Colorado","POLYGON ((-104.82995 39.69662, -104.83047 39.6..."


In [27]:
type(census_tracts_geo_df)

geopandas.geodataframe.GeoDataFrame

In [28]:
census_tracts_geo_df['area'] = census_tracts_geo_df.area

In [29]:
census_tracts_geo_df

,Tract_ID,NAME,geometry,area
0,8043979000,"Census Tract 9790, Fremont County, Colorado","POLYGON ((-105.88353 38.32817, -105.88385 38.3...",0.269868
1,8045951600,"Census Tract 9516, Garfield County, Colorado","POLYGON ((-107.43227 39.57173, -107.43309 39.5...",0.157182
2,8069002803,"Census Tract 28.03, Larimer County, Colorado","POLYGON ((-105.57906 40.33209, -105.57910 40.3...",0.060308
3,8125963200,"Census Tract 9632, Yuma County, Colorado","POLYGON ((-102.80355 39.99325, -102.80359 39.9...",0.282556
4,8069002401,"Census Tract 24.01, Larimer County, Colorado","POLYGON ((-105.22073 40.79379, -105.22117 40.7...",0.110759
...,...,...,...,...
1244,8005081000,"Census Tract 810, Arapahoe County, Colorado","POLYGON ((-104.82687 39.71730, -104.82687 39.7...",0.000249
1245,8005080300,"Census Tract 803, Arapahoe County, Colorado","POLYGON ((-104.86015 39.68204, -104.85898 39.6...",0.000265
1246,8005080400,"Census Tract 804, Arapahoe County, Colorado","POLYGON ((-104.84359 39.65845, -104.84377 39.6...",0.000254
1247,8005080800,"Census Tract 808, Arapahoe County, Colorado","POLYGON ((-104.82995 39.69662, -104.83047 39.6...",0.000096


In [14]:
census_tracts.shape

(1249, 3)

In [17]:
env_effects = pd.read_sql("SELECT * FROM environmental_effects;", con)
env_effects.head()

,index,Tract_ID,PTSDF,PTSDF_rank,PRMP,PRMP_rank,PWDIS,PWDIS_rank,PNPL,PNPL_rank
0,0,08001007801,1.001907,78.462770,0.729229,74.619696,0.001463,40.832666,0.120817,73.178543
1,1,08001007802,1.223296,81.505204,0.733042,74.779824,0.000661,35.468375,0.102412,66.132906
2,2,08001007900,1.394943,84.307446,1.737404,92.073659,0.000785,36.909528,0.122560,73.738991
3,3,08001008000,1.682376,87.750200,1.607611,90.712570,0.001586,41.473179,0.108019,69.015212
4,4,08001008100,2.510113,93.274620,1.208683,85.508407,0.001666,41.793435,0.105026,67.654123


In [18]:
env_effects.shape

(3747, 10)

In [11]:
sensitive_pop = pd.read_sql("SELECT * FROM sensitive_population;", con)
sensitive_pop.head()

,index,Tract_ID,asthma,asthma_rank,low_bw,low_bw_rank,heart_disease,heart_disease_rank
0,0,08043979000,527.94,35.024155,7.29,56.025105,5.9,97.580645
1,1,08045951600,266.94,8.132045,3.61,4.644351,3.0,61.491935
2,2,08069002803,0.00,0.483092,0.00,0.543933,0.5,0.645161
3,3,08125963200,350.95,14.734300,4.64,12.845188,5.8,97.379032
4,4,08069002401,1002.35,85.426731,0.00,0.543933,4.7,93.508065


In [16]:
sensitive_pop.shape

(1249, 8)